<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/english_to_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Arun\Downloads\1000K IITM.csv\1000K IITM.csv")
df

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
1561836,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...
1561838,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...
1561839,खाद्य और जल सुरक्षा; पर्यावरण की दृष्टि से वहन...,्त है। Issues such as food and water security;...


In [ ]:
df=df[:50000]
df.fillna('',inplace=True)

C:\Users\Arun\AppData\Local\Temp\ipykernel_4692\1990341210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('',inplace=True)


In [ ]:
hindi=df["hindi"].to_list()
eng=df["english"].to_list()

In [ ]:
from nltk import word_tokenize
def preprocess(text):
    alltokens=[word_tokenize(word) for word in text]
    return alltokens
hindi_tokens=preprocess(hindi)
eng_tokens=preprocess(eng)

In [ ]:
hindi_tokens_list=[]
eng_tokens_list=[]
for row in hindi_tokens:
    for word in row:
        hindi_tokens_list.append(word)
for row in eng_tokens:
    for word in row:
        eng_tokens_list.append(word)

In [ ]:
from collections import Counter
def counter(text):
    counts=Counter(text)
    return counts
hindi_word_counts=counter(hindi_tokens_list)
english_word_counts=counter(eng_tokens_list)

In [ ]:
hindi_vocab={"<pad>":0,"<unk>":1}
english_vocab={"<pad>":0,"<unk>":1}
for word in hindi_word_counts:
    if word not in hindi_vocab:
        hindi_vocab[word]=len(hindi_vocab)
for word in english_word_counts:
    if word not in english_vocab:
        english_vocab[word]=len(english_vocab)

In [ ]:
def process(text,dict):
    sequence=[]
    for row in text:
        index=[dict.get(word,dict["<unk>"]) for word in row]
        sequence.append(index)
    print(sequence)
    return sequence
y_sequence=process(hindi_tokens,hindi_vocab)
x_sequence=process(eng_tokens,english_vocab)

[[2, 3, 4, 5, 6, 7, 8, 9], [10, 5, 11], [12, 13, 14, 15, 16, 17, 18], [19, 13, 14, 15, 16, 17, 18], [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30], [31, 4, 32, 33], [34, 35, 36, 37, 38, 4, 39, 40, 32, 41, 22, 31], [42, 36, 43, 38, 14, 44, 4, 32, 45], [32, 46, 47, 42, 7, 44, 48, 49], [50, 14, 44, 4, 32, 45], [32, 28, 29, 50, 7, 44, 48, 51], [52, 53], [54, 40, 55, 56, 28, 29, 57, 58, 59, 60, 61, 36, 62, 38, 63, 64, 45], [65, 66, 4, 67], [68], [69], [70], [71, 72], [54, 40, 73, 47, 74, 26, 75, 76, 14, 15, 77, 72], [78, 79], [80, 4, 79, 45, 36, 81, 82, 38], [83, 4, 84, 36, 85, 81, 38], [86], [73, 47, 3], [73, 47, 74], [87], [78, 79], [73, 47, 88, 48, 89, 26, 90, 91, 92, 93, 94, 58, 95, 96, 30], [97, 98, 78, 4, 32, 99], [78, 100, 101, 102], [78, 103, 104], [105, 106, 107], [108], [109], [34], [110, 111, 45], [110, 36, 81, 112, 38], [113], [114, 115], [116], [3], [117], [118, 58, 118], [119, 120], [121], [122], [123], [124], [125], [126, 120], [127], [128], [129], [130], [131], [132], [120

In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class positional_encoding(nn.Module):
    def __init__(self,max_len,d_model):
        super().__init__()

        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len)
        div_term=torch.exp(torch.arange(0,d_model,2)*(-math.log(10000.0)/d_model))
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0)
        self.register_buffer("pe",pe)


    def forward(self,x):
        return x+x[:,:x.size(1)]

In [ ]:
class attention(nn.Module):
    def __init__(self,d_model,num_heads):
        super().__init__()
        assert d_model%num_heads==0
        self.d_k=d_model//num_heads
        self.num_heads=num_heads

        self.q_linear=nn.Linear(d_model,d_model)
        self.k_linear=nn.Linear(d_model,d_model)
        self.v_linear=nn.Linear(d_model,d_model)
        self.out=nn.Linear(d_model,d_model)

    def forward(self,q,k,v,mask=None):
        batch=q.size(0)
        Q=self.q_linear(q).view(batch,-1,self.num_heads,self.d_k).transpose(1,2)
        K=self.q_linear(k).view(batch,-1,self.num_heads,self.d_k).transpose(1,2)
        V=self.q_linear(v).view(batch,-1,self.num_heads,self.d_k).transpose(1,2)

        scores=torch.matmul(Q,K.transpose(-2,-1))/math.sqrt(self.d_k)
        if mask is not None:
            scores=scores.masked_fill(mask==0,float("-inf"))
        attn=torch.softmax(scores,dim=-1)
        output=torch.matmul(attn,V)
        output=output.transpose(1,2).contiguous().view(batch,-1,self.num_heads*self.d_k)
        return self.out(output)


In [ ]:
class feedforward(nn.Module):
    def __init__(self,d_model,neurons=2048,drop=0.1):
        super().__init__()
        self.features=nn.Sequential(
            nn.Linear(d_model,neurons),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(neurons,d_model)
            )
    def forward(self,x):
        return self.features(x)

In [ ]:
class encoder(nn.Module):
    def __init__(self,d_model,num_heads,neurons=2048,drop=0.1):
        super().__init__()
        self.attn=attention(d_model,num_heads)
        self.ff=feedforward(d_model,neurons,drop)
        self.norm=nn.LayerNorm(d_model)
        self.norm2=nn.LayerNorm(d_model)
        self.dropout=nn.Dropout(drop)
    def forward(self,x,mask=None):
        attn=self.attn(x,x,x,mask)
        x=self.norm(x+self.dropout(attn))
        ff_out=self.ff(x)
        x=self.norm2(x+self.dropout(ff_out))
        return(x)

In [ ]:
class decoder(nn.Module):
    def __init__(self,d_model,num_heads,neurons=2048,drop=0.1):
        super().__init__()
        self.attn=attention(d_model,num_heads)
        self.cross_attn=attention(d_model,num_heads)
        self.ff=feedforward(d_model,neurons,drop)
        self.norm1=nn.LayerNorm(d_model)
        self.norm2=nn.LayerNorm(d_model)
        self.norm3=nn.LayerNorm(d_model)
        self.dropout=nn.Dropout(drop)
    def forward(self,x,enc_out,src_mask=None,tgt_mask=None):
        attn=self.attn(x,x,x,tgt_mask)
        x=self.norm1(x+self.dropout(attn))
        cross_attn=self.cross_attn(x,enc_out,enc_out,src_mask)
        x=self.norm2(x+self.dropout(cross_attn))
        ff_out=self.ff(x)
        x=self.norm3(x+self.dropout(ff_out))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, num_heads=8, num_layers=6, neurons=2048, max_len=100 ,dropout=0.1):
        super().__init__()
        self.src_embed = nn.Embedding(src_vocab, d_model)
        self.tgt_embed = nn.Embedding(tgt_vocab, d_model)
        self.pos_enc = positional_encoding(max_len,d_model)

        self.encoder_layers = nn.ModuleList([encoder(d_model, num_heads, neurons, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([decoder(d_model, num_heads, neurons, dropout) for _ in range(num_layers)])

        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src = self.pos_enc(self.src_embed(src))
        tgt = self.pos_enc(self.tgt_embed(tgt))

        for layer in self.encoder_layers:
            src = layer(src, src_mask)

        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)

        return self.fc_out(tgt)

In [ ]:
import torch.nn as nn
class early_stopping(nn.Module):
    def __init__(self,min_delta,patience):
        super().__init__()
        self.min_delta=min_delta
        self.partience=patience
        self.early_stop=False
        self.best_loss=float("inf")
        self.counter=0

    def __call__(self,val_loss):
        if self.min_delta >= val_loss-self.best_loss:
            self.counter=0
            self.best_loss=val_loss
        else:
            self.counter+=1
            if self.counter>=self.partience:
                self.early_stop=True

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def tensor(tensor):
    new_tensor= [torch.tensor(row_tensor,dtype=torch.long) for row_tensor in tensor]
    padded_tensor=pad_sequence(new_tensor)
    return padded_tensor

x_sequence=tensor(x_sequence)
y_sequence=tensor(y_sequence)

In [ ]:
tgt_vocab=len(hindi_vocab)
src_vocab=len(english_vocab)
print(f"src_vocab : {src_vocab},tgt_vocab: {tgt_vocab}")

src_vocab : 5149,tgt_vocab: 5170


In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
model=Transformer(src_vocab=src_vocab,tgt_vocab=tgt_vocab,d_model=512,num_layers=6,num_heads=8,max_len=256).to(device)
model_loss=nn.CrossEntropyLoss(ignore_index=hindi_vocab["<pad>"])
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode="min",factor=0.5,patience=2)

In [ ]:
y_sequence=y_sequence[:135,:2000]
x_sequence=x_sequence[:,:2000]
print(x_sequence.shape)
print(y_sequence.shape)

torch.Size([135, 2000])
torch.Size([135, 2000])


In [ ]:
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
dataset=TensorDataset(x_sequence,y_sequence)
training_dataset,validation_data=train_test_split(dataset,test_size=0.2,random_state=7)
training_dataset_loader=DataLoader(training_dataset,batch_size=1)
validation_dataset_loader=DataLoader(validation_data,batch_size=1)

In [ ]:
from torch.amp import GradScaler,autocast
epochs=50
stopper=early_stopping(1e-4,4)
training_loss_store=[]
validation_loss_store=[]
training_accuracy_store=[]
validation_accuracy_store=[]
scaler=GradScaler()

for epoch in range(epochs):
    running,total,correct=0,0,0
    for x,y in training_dataset_loader:
        x,y=x.to(device),y.to(device)

        optimizer.zero_grad()
        with autocast("cuda"):
            output=model(x,y)
            output=output.view(-1,output.shape[-1])
            loss=model_loss(output,y)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        running+=loss.item()
        _,pred=torch.max(output,1)
        correct+=(pred==y).sum().item()
        total+=y.size(0)
        # print(torch.isnan(pred).any(), torch.isnan(y).any())

    training_loss=running/len(y)
    training_acc=correct/total
    training_loss_store.append(training_loss)
    training_accuracy_store.append(training_acc)

    model.eval()
    vrunning,vtotal,vcorrect=0,0,0
    for x,y in validation_dataset_loader:
        x,y=x.to(device),y.to(device)

        optimizer.zero_grad()
        with autocast("cuda"):
            voutput=model(x,y)
            voutput=voutput.view(-1,voutput.shape[-1])
            y=y.view(-1)
            vloss=model_loss(voutput,y)

        vrunning+=vloss.item()
        _,vpred=torch.max(voutput,1)
        vcorrect+=(vpred==y).sum().item()
        vtotal+=y.size(0)
    val_loss=vrunning/len(y)
    val_acc=vcorrect/vtotal
    validation_loss_store.append(val_loss)
    validation_accuracy_store.append(val_acc)

    print(f"epoch: {epoch+1} training_loss: {training_loss:.4f} training acc:{training_acc:.2f} validation_loss: {val_loss:.4f} validation_acc: {val_acc:.2f} ")
    stopper(val_loss)
    if stopper.early_stop:
        print("early_stopping triggered")
        break

In [ ]:
translate="hey its me how are you"
transalate=preprocess(translate)
transalate=process(transalate)
max_len=2000
if len(transalate)<=max_len:
    transalate+=[english_vocab["<pad>"]*max_len-len(transalate)]
else:
    transalate=transalate[:,:max_len]
input_new_tensor=torch.tensor(transalate).unsqueeze(0).to(device)
output=model(input_new_tensor)
print(output)